In [1]:
# -*- coding: utf-8 -*-  
# Опыты так сказать с:

In [149]:
import sys
import os
import numpy as np
import doc2words
import docreader
from struct import pack, unpack, calcsize
from hashlib import md5, algorithms

from compress import Varbyte, Simple9

In [186]:
DATA_PATH = './dataset'
FILES = os.listdir(DATA_PATH)
FILES = [DATA_PATH + '/' + _file for _file in sorted(FILES[:1])]
FILES

['./dataset/lenta.ru_4deb864d-3c46-45e6-85f4-a7ff7544a3fb_01.gz']

In [187]:
reader = docreader.DocumentStreamReader(FILES)

In [188]:
%%time
_dict = {}
for docID, doc in enumerate(reader):
    url = doc.url # не буду делать DocID <=> doc.url
    body = doc.body # Там нет ни в одном документе тела
    text = doc.text
    words = doc2words.extract_words(text)
    for word in words:
        word = unicode(word).encode('utf8')
        termID = int(md5(word).hexdigest(), 16) # get int value of hash
        if termID in _dict:
            if docID not in _dict[termID]:
                _dict[termID] = np.concatenate((_dict[termID], [docID]))
        else:
            _dict[termID] = np.asarray([docID])
    if docID == 10: break

CPU times: user 26.9 ms, sys: 5.39 ms, total: 32.3 ms
Wall time: 31.5 ms


In [189]:
for i, _hash in enumerate(_dict):
    print str(_hash) + ': ' + str(_dict[_hash])
    if i == 20: break

311878378528305137774189393495128081847: [6]
220798841255043928211217796664636418996: [10]
307450413009592169440822615273533403989: [4]
132472370778067497765503863393117737714: [ 0  1  2  3  7 10]
206652403000100525148608061927411431115: [1]
299029326086221060415830536176903133998: [1]
7336478196583621385623378817424033646: [ 0  2  3  5 10]
63548883548822514986286344258916628542: [1]
69986279194967900595372218141405592701: [2]
22935343108867884981181042812720467261: [3]
27345436137887647652709782145180837294: [7]
284420083180970213176816728746115774957: [3]
222372557009841293096181427623663036811: [10]
117148122059749065314445825367308118613: [10]
77242452832178521887592784196652095852: [6]
291439818284753552045801114852863196068: [ 8 10]
35999664102191865165066689740873200004: [3]
302733654877821793922912767178351320795: [1]
54856839013676489318636616771634651576: [7]
190885061693855529773532119764352883513: [7]
42306242971694659741827129090935981113: [7]


In [199]:
class Indexer():
    """
    Makes, Compresses, and Saves dict
    """
    def __init__(self, compressor='varbyte', hash_='md5', coding='utf8'):
        """
        compressors: ['varbyte', 'simple9']
        hash_: ['md5'] (hash_ must have hexdigest() attribute)
        coding: any (tested only with utf8)
        """
        
        self.dict = {}
        self.compressed_dict = None

        if compressor == 'varbyte': self.compressor = Varbyte()
        elif compressor == 'simple9': self.compressor = Simple9()
        else: raise RuntimeError('Compressor: {} not found'.format(compressor))
            
        if hash_ == 'md5': self.hash = md5
        else: raise RuntimeError('Hash: {} not found'.format(hash_))
            
        self.coding = coding
     
    def make_dict(self, doc_reader):
        for docID, doc in enumerate(doc_reader):
            if docID % 128 != 0: continue
            url = doc.url # не буду делать DocID <=> doc.url
            body = doc.body # Там нет ни в одном документе тела
            text = doc.text
            words = doc2words.extract_words(text)
            for word in words:
                word = unicode(word).encode(self.coding)
                termID = int(self.hash(word).hexdigest(), 16) # get int value of hash
                if termID in self.dict:
                    if docID not in self.dict[termID]:
                        self.dict[termID] = np.concatenate((self.dict[termID], [docID]))
                else:
                    self.dict[termID] = np.asarray([docID])
        return
    
    def compress(self):
        if self.compressed_dict is not None:
            print >> sys.stderr, 'Dictionary was already compressed'
        self.compressed_dict = {}
        for termID in self.dict:
            self.compressed_dict[termID] = self.compressor.pack(self.dict[termID])
        # Удалить словарь, чтобы не занимал память?
        self.dict = {}
        
    def save(self, OUTPUT_FILE='index.dat'):
        if self.compressed_dict is None:
            raise RuntimeError('Compress dictionaty before saving')
        output = open(OUTPUT_FILE, 'wb')
        for termID in self.compressed_dict:
            array = self.compressed_dict[termID]
            # 128bits for termID, 64bist for array_size and array
            output.write(pack('QQQ%sB' % array.shape, termID / 2**64, termID % 2**64, array.shape[0], *array))
        output.close()
        
    def _save(self, OUTPUT_FILE='index.dat'):
        if self.compressed_dict is None:
            raise RuntimeError('Compress dictionaty before saving')
        output = open(OUTPUT_FILE, 'wb')
        for termID in self.compressed_dict:
            array = self.compressed_dict[termID]
            output.write(pack('QQ%sB' % array.shape, termID / 2**64, termID % 2**64,  *array))
        output.close()

In [200]:
Idxr = Indexer()
%time Idxr.make_dict(reader)

CPU times: user 86 ms, sys: 4.45 ms, total: 90.4 ms
Wall time: 89.4 ms


In [201]:
Idxr.compress()
Idxr.save()